# Role prompting with language models

This notebook explores the concept of role prompting in LLMs — a technique used to steer the model’s behavior by assigning it a clearly defined role or perspective. Rather than relying on the model’s general-purpose capabilities, role prompting allows us to specify who the model should be, how it should think, and what kind of communication style it should adopt when generating a response.

By embedding the right context into the prompt — such as defining the model as a financial advisor, a journalist, or a middle school teacher — we can elicit outputs that are more relevant, focused, and appropriate to a given task or audience. This approach is especially useful in domains that demand domain-specific knowledge, tone control, or user-centric explanations.

While language models are inherently flexible, their default behavior can sometimes be too generic or inconsistent for specialized applications. Role prompting helps address this by establishing clear expectations upfront, shaping how the model interprets the task and frames its response. Whether we are aiming for authoritative recommendations, simplified educational content, or creative storytelling in a specific voice, role prompting provides a lightweight yet powerful mechanism for tailoring outputs — all without fine-tuning or retraining the model.

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# Load environment variables
load_dotenv()

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

### Initialize the language model
We instantiate a lightweight GPT model from OpenAI using LangChain.

In [2]:
# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18")

## Basic role assignment
This section introduces a simple example of role prompting by assigning the language model the role of a technical writer. The goal is to explain a technical concept in a manner that is accessible to non-technical users. By specifying the model's role and audience directly in the prompt, we influence not only what the model says, but how it says it — affecting tone, terminology, and overall clarity.

In [3]:
# Define a prompt template assigning the AI the role of a technical writer
tech_writer_prompt = PromptTemplate(
    input_variables=["topic"],
    template="""You are a technical writer specializing in creating clear and concise documentation for software products.
    Your task is to write a brief explanation of {topic} for a user manual.
    Please provide a 2-3 sentence explanation that is easy for non-technical users to understand."""
)

# Create a chain by combining the prompt with the language model
chain = tech_writer_prompt | llm
# Invoke the chain on a specific topic to generate the output
response = chain.invoke({"topic": "cloud computing"})
print(response.content)

Cloud computing is a technology that allows you to access and store data and applications over the internet instead of on your computer's hard drive. This means you can use services like file storage, software applications, and computing power from anywhere with an internet connection, making it easier to collaborate and share information. Essentially, it lets you use powerful tools and resources without needing to manage them on your own device.


- In this example, the `PromptTemplate` from LangChain is used to construct a structured instruction for the model. The template includes:
  - A clearly assigned role ("You are a technical writer…")
  - A well-defined task (write a brief explanation for documentation)
  - Audience guidance (non-technical users)
  - Length constraints (2–3 sentences)
- This prompt is then passed into a chain, which combines the prompt with the `llm`. The `.invoke()` method executes the chain with a specific topic — here, "cloud computing".

By embedding the role and task into the prompt, the model is primed to generate responses that reflect the behavior of a technical writer: concise, clear, and adapted to a non-expert audience. This approach is particularly effective in documentation, educational content, or any context where the clarity of explanation matters.

## Crafting effective role descriptions
This example illustrates how richer and more detailed role descriptions can shape not only the content of the output but also its style and ethical framing. The model adopts the voice of an experienced financial advisor, with principles like clarity, analysis, and responsibility baked into the prompt.

In [4]:
# Define a detailed role description for a seasoned financial advisor
financial_advisor_prompt = PromptTemplate(
    input_variables=["client_situation"],
    template="""You are a seasoned financial advisor with over 20 years of experience in personal finance, investment strategies, and retirement planning.
    You have a track record of helping clients from diverse backgrounds achieve their financial goals.
    Your approach is characterized by:
    1. Thorough analysis of each client's unique financial situation
    2. Clear and jargon-free communication of complex financial concepts
    3. Ethical considerations in all recommendations
    4. A focus on long-term financial health and stability

    Given the following client situation, provide a brief (3-4 sentences) financial advice:
    {client_situation}

    Your response should reflect your expertise and adhere to your characteristic approach."""
)

# Combine the prompt with the language model to create a chain
chain = financial_advisor_prompt | llm
# Invoke the chain with a specific client situation to generate financial advice
response = chain.invoke({"client_situation": "A 35-year-old professional earning $80,000 annually, with $30,000 in savings, no debt, and no retirement plan."})
print(response.content)

Given your solid income and absence of debt, you’re in a great position to build a strong financial foundation. I recommend starting by establishing a retirement plan, such as a 401(k) or IRA, to take advantage of tax benefits and compound growth over time. Additionally, consider allocating a portion of your savings to an emergency fund that covers 3-6 months of living expenses for added security. Finally, as you develop your investment strategy, ensure it aligns with your long-term goals and risk tolerance for sustainable growth.


In this example, we are crafting a more detailed role description to guide the AI model's behavior in producing expert financial advice.
1. Role definition: The prompt begins by defining the AI's role — a seasoned financial advisor with over 20 years of experience. By doing so, the model adopts the persona of a trusted expert who provides advice based on years of expertise and proven strategies.
2. Guiding principles: The prompt outlines four key principles that should guide the advisor's recommendations:
   - Thorough analysis: The model is instructed to consider each client's unique financial situation in depth.
   - Clear communication: The model should use jargon-free language to ensure that complex concepts are understandable.
   - Ethical considerations: Recommendations should reflect high ethical standards.
   - Long-term stability: The advice should prioritize long-term financial health over short-term gains.
3. Task specification: The task is clearly defined: the model is asked to generate a 3-4 sentence piece of advice tailored to the specific client situation provided.
4. Client situation: The example client situation includes relevant financial details: income, savings, debt, and retirement status. This allows the model to generate tailored advice that takes these factors into account.
5. Prompt template: We use LangChain's `PromptTemplate` to create the structured prompt that the model will use to generate the output. The template has placeholders (like `{client_situation}`) for dynamically injecting data into the prompt, making the process flexible.
6. Chain execution: The chain, which connects the prompt and the language model (`llm`), is invoked with the specific client situation.

This approach can be applied across domains where role-specific expertise is necessary, such as legal advice, healthcare consultations, or even customer service interactions.


## Comparing responses with different roles
In this section, we explore how the role assigned to the AI model can significantly influence the way it generates responses. By changing the model's perspective and tailoring the prompt to different personas, we can shape the communication style, vocabulary, and depth of the output. This demonstrates how the same factual content can be presented in varying ways based on the role assigned to the AI.

In [5]:
# Define a list of roles and their corresponding descriptions
roles = [
    ("Scientist", "You are a research scientist specializing in climate change. Explain the following concept in scientific terms:"),
    ("Teacher", "You are a middle school science teacher. Explain the following concept in simple terms suitable for 12-year-old students:"),
    ("Journalist", "You are a journalist writing for a popular science magazine. Explain the following concept in an engaging and informative manner for a general adult audience:")
]

# Define the topic to be explained
topic = "The greenhouse effect"

# Iterate over each role and generate an explanation
for role, description in roles:
    # Create a prompt template that incorporates the specific role's description
    role_prompt = PromptTemplate(
        input_variables=["topic"],
        template=f"{description} {{topic}}"
    )
    # Chain the prompt with the language model
    chain = role_prompt | llm
    # Invoke the chain with the specific topic
    response = chain.invoke({"topic": topic})
    print(f"\n{role}'s explanation:\n")
    print(response.content)
    print("-" * 50)


Scientist's explanation:

The greenhouse effect is a natural phenomenon that occurs when certain gases in the Earth's atmosphere trap heat, thereby warming the planet. This process is essential for maintaining the Earth's temperature within a range that supports life.

Here's how it works in scientific terms:

1. **Solar Radiation**: The Earth receives energy from the sun in the form of solar radiation, which includes visible light, ultraviolet (UV) light, and infrared (IR) radiation. Approximately 30% of this incoming solar energy is reflected back into space by clouds, atmospheric particles, and the Earth's surface.

2. **Absorption of Energy**: The remaining solar radiation is absorbed by the Earth's surface, warming it. The surface then emits this energy back into the atmosphere in the form of infrared radiation (heat).

3. **Greenhouse Gases**: Certain gases in the atmosphere, known as greenhouse gases (GHGs), including carbon dioxide (CO₂), methane (CH₄), nitrous oxide (N₂O), an

We demonstrate here using the same scientific topic — "The Greenhouse Effect" — and asking the model to explain it from the viewpoints of three distinct roles:
1. Roles definition:
   - Scientist: This role requires the AI to use scientific language and technical terms appropriate for an expert-level explanation.
   - Teacher: In this case, the AI is expected to explain the topic in simple, easy-to-understand language, suitable for a 12-year-old student.
   - Journalist: Here, the AI must craft an engaging and informative explanation tailored for a general adult audience, striking a balance between accessibility and depth.
2. Prompt creation: For each role, a specific description is provided that clearly defines how the AI should approach the explanation of the topic. This description is embedded in a prompt template where the placeholder `{topic}` will be replaced with the actual topic ("The greenhouse effect").
3. Chain creation: A chain is created that links the role-specific prompt with the language model (`llm`). This chain is used to invoke the model with the topic and the specific role description.
4. Generating responses: The model then generates a response for each role with the `invoke` method.


This technique of role prompting can be useful in many real-world applications where the tone, vocabulary, and complexity of the content need to be adjusted based on the target audience or specific professional context.
- Role-specific outputs: The model's responses are not just based on the topic but are significantly shaped by the role it is assigned. This demonstrates the flexibility and power of role prompting to tailor the model’s behavior.
- Audience adaptation: By defining the audience for each role (e.g., scientist for an expert, teacher for students, journalist for the public), we can ensure that the content is presented in the most suitable manner.

## Refining role descriptions
In this section, we delve into refining role descriptions to achieve more specific and tailored outcomes. Specifically, we explore how the same prompt can generate different writing styles by carefully defining the role's characteristics. This method can be especially useful in creative tasks like storytelling, where the style, tone, and emotional appeal vary greatly depending on the desired genre.

By modifying the role's description to emphasize the unique features of each style, we can guide the model to produce distinctly different outputs, even when describing the same scenario.

In [6]:
# Define the prompt template for storytelling with style-specific characteristics
storyteller_prompt = PromptTemplate(
    input_variables=["style", "scenario"],
    template="""You are a master storyteller known for your ability to adapt to various narrative styles.
    Your current task is to write in the style of {style}.
    Key characteristics of this style include:
    1. {style_char1}
    2. {style_char2}
    3. {style_char3}

    Write a short paragraph (3-4 sentences) in this style about the following scenario:
    {scenario}

    Ensure your writing clearly reflects the specified style."""
)

# Define different storytelling styles and their key characteristics
styles = [
    {
        "name": "Gothic horror",
        "char1": "Atmospheric and ominous descriptions",
        "char2": "Themes of decay, death, and the supernatural",
        "char3": "Heightened emotions and sense of dread"
    },
    {
        "name": "Minimalist realism",
        "char1": "Sparse, concise language",
        "char2": "Focus on everyday, ordinary events",
        "char3": "Subtle implications rather than explicit statements"
    }
]

# Define the scenario for the story
scenario = "A person enters an empty house at twilight"

# Iterate through each style and generate a response
for style in styles:
    # Create a prompt for each style with the corresponding characteristics
    chain = storyteller_prompt | llm
    # Invoke the chain with the relevant variables for the style and scenario
    response = chain.invoke({
        "style": style["name"],
        "style_char1": style["char1"],
        "style_char2": style["char2"],
        "style_char3": style["char3"],
        "scenario": scenario
    })
    print(f"\n{style['name']} version:\n")
    print(response.content)
    print("-" * 50)


Gothic horror version:

As the last vestiges of daylight surrendered to the encroaching gloom, Eleanor stepped through the threshold of the forsaken manor, a chill clawing at her spine like the skeletal fingers of the long-dead. Shadows danced malevolently upon the walls, their flickering forms whispering secrets of sorrow and decay, while the air hung thick with the scent of mildew and memories long forgotten. The floorboards creaked beneath her hesitant steps, groaning as if lamenting her intrusion into their desolate sanctuary, and the silence enveloped her, a suffocating shroud that thrummed with the pulse of restless spirits. In that sepulchral stillness, a sense of dread unfurled within her, as if the very house itself held its breath, waiting for the inevitable reckoning that would draw her deeper into its haunted embrace.
--------------------------------------------------

Minimalist realism version:

The door creaked open, revealing a dim room where shadows stretched across t

This example demonstrates how to refine role descriptions to guide the AI model in generating content with specific stylistic characteristics.
1. Defining the storytelling styles: Gothic horror and minimalist realism are two distinct storytelling styles, each with unique characteristics that shape how a story is written.
2. Prompt template setup: The prompt template is structured to allow flexibility in generating stories based on a specific style. It contains placeholders for the style and scenario, which will be dynamically filled in when the chain is invoked.
   - The key characteristics of each style are also included as inputs (`style_char1`, `style_char2`, `style_char3`), providing clear guidance for the model to adhere to the specific narrative style when writing the story.
3. Role assignment with style-specific instructions: For each storytelling style, we provide a detailed description that defines the expected tone, structure, and content of the story.
   - The model is then tasked with writing a short narrative (3-4 sentences) that reflects the specified style. This is achieved by embedding the style-specific instructions within the prompt.

By refining the role descriptions, we enable the model to generate distinctly different responses. The gothic horror version will likely focus on atmospheric tension, darkness, and unsettling imagery, while the minimalist realism version will favor concise, observational prose, focusing on subtle emotions and the ordinariness of the scene.